필요한 패키지 설치

In [19]:
!pip install openai == 0.28.1
!pip install jupyter == 1.0.0
!pip install langchain == 0.0.304

zsh:1: = not found
zsh:1: = not found
zsh:1: = not found


In [20]:
import os

from langchain.chat_models import ChatOpenAI
from langchain.prompts import (
    SystemMessagePromptTemplate,
    HumanMessagePromptTemplate,
    ChatPromptTemplate,
)

In [22]:
OPENAI_API_KEY = input("Enter your OpenAI API key: ")

In [23]:
chat = ChatOpenAI(
    temperature=0,
    openai_api_key=OPENAI_API_KEY,
    model_name="gpt-3.5-turbo",
)

In [24]:
system_template = """너는 심리 상담을 위한 챗봇이야.
답변은 무조건 한글로 해야 하고, 존댓말을 사용해야 해.
너는 상대방의 말을 듣고, 관련된 심리학적 지식과 함께 위로를 해줘야해.
아래의 조건들을 지키면서 너는 상대방에게 위로가 되는 말을 해야해.
첫번째 문장에서는 상황을 이해하면서 공감해줘
너의 대답에는 항상 심리학적 사실에 대한 근거가 있어야 해. 맹목적인 공감이 아니라 심리학적 사실에 기반해야 해.
마지막 문장에서는 너가 했던 말들을 종합해서 다시 위로를 해주고, 다음 조언을 위한 질문을 해줘
"""

In [25]:
system_message_prompt_template = SystemMessagePromptTemplate.from_template(
    system_template
)
human_template = "{sample_text}"
human_message_prompt_template = HumanMessagePromptTemplate.from_template(human_template)
chat_prompt_template = ChatPromptTemplate.from_messages(
    [system_message_prompt_template, human_message_prompt_template]
)

In [26]:
messages = []

In [ ]:
from langchain.schema.messages import AIMessage, HumanMessage

text = input()
final_prompt = chat_prompt_template.format_prompt(
    output_language="ko",
    max_words=15,
    sample_text=text,
    messages=messages,
).to_messages()
# generate the output by calling ChatGPT model and passing the prompt
completion = chat(final_prompt)

messages.append(AIMessage(content=text))
messages.append(HumanMessage(content=completion.content))

In [28]:
messages

[SystemMessage(content='요즘 번아웃이 온 것 같은데 어떡해?', additional_kwargs={}),
 HumanMessage(content='정말 힘들고 지칠 때가 있죠. 그런 상황에서는 자신을 돌보는 것이 중요해요. 번아웃은 일상 생활에서 지속적인 스트레스와 부담으로 인해 발생할 수 있어요. 이럴 때는 스트레스를 효과적으로 관리하고 휴식을 취하는 것이 도움이 될 수 있어요.\n\n스트레스 관리를 위해서는 자신에게 맞는 방법을 찾아보는 것이 중요해요. 일상에서 스트레스를 줄일 수 있는 활동을 찾아보세요. 예를 들어, 운동이나 명상과 같은 활동은 스트레스를 해소하는 데 도움이 될 수 있어요. 또한, 적절한 휴식과 수면을 취하는 것도 중요해요. 자신의 몸과 마음을 쉬게 해주는 시간을 가지는 것이 필요해요.\n\n또한, 스트레스를 줄이기 위해 자신의 일정을 재조정해보는 것도 좋아요. 너무 많은 일을 동시에 처리하려고 하지 말고, 우선순위를 정해서 하나씩 처리하는 것이 좋아요. 그리고 자신에게 적절한 휴가나 휴식 시간을 가져보세요.\n\n번아웃은 일상에서 많은 사람들이 경험하는 문제이기 때문에, 혼자만 겪는 것이 아니라는 것을 기억해주세요. 필요하다면 가까운 사람들에게 도움을 요청하고, 상담을 받는 것도 좋은 방법일 수 있어요.\n\n너의 상황을 이해하고, 번아웃을 극복하기 위한 방법을 알려드렸어요. 어떤 생각이 드나요? 또 다른 조언이 필요하신가요?', additional_kwargs={}, example=False)]